# GET DATA FROM NYT BESTSELLER API 

The dataset is gathered from the New York Times Books API for 'Combined Print and E-book' Fiction best sellers from December 19, 2021 to December 18, 2022.

The API can be found here: https://developer.nytimes.com/

Collected data includes the book title, author, the publisher, the isbn, the contributor, the date of the best seller list, the rank (this week and last week), number of weeks on the list, and the price.

In [ ]:
import requests, math, datetime, time
import os
import json
import csv
import pprint

In [ ]:
#API KEYS
API_KEY = "api_key" #API KEYS

#PARAMETERS
LIST = "/Combined%20Print%20and%20E-Book%20Fiction"

BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists/"

#PARAMETERS
LIST = "/Combined%20Print%20and%20E-Book%20Fiction"

BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists/"

In [ ]:
def parse_books(book_list, date):
    books = []
    for book in book_list['results']['books']:
        dic = {}
        dic['title'] = book['title']
        dic['author'] = book['author']
        dic['price'] = float(book['price'])
        dic['publisher'] = book['publisher']
        dic['isbn'] = book['primary_isbn10']
        dic['contributor'] = book['contributor']
        dic['date'] = date  
        dic['rank'] = book['rank']
        dic['rank_last_week'] = book['rank_last_week']
        dic['weeks_on_list'] = book['weeks_on_list']      
        books.append(dic)
    return(books) 

In [ ]:
    bestsellers = []
    num_days_to_get = 365 #you can use up to 1000 api calls a day so max=1000
    total = 0
    
    #create a directory to save data in csv files
    if not os.path.exists('books'):
        os.mkdir('books')
    
    for i in range(0,num_days_to_get):
        day = datetime.datetime.now() - datetime.timedelta(days=i)  
        date = str(day.date())
        print('Dowloading bestseller books from....'+ date)
        
        #send api request
        url = BOOKS_ROOT+date+LIST+'.json?api-key='+API_KEY
        #send api request
        response = requests.get(url)
        time.sleep(6)
        
        if (response.status_code == 200):
            results = response.json()
            books = parse_books(results, date)
            bestsellers.extend(books)
            df = pd.DataFrame(bestsellers) 
            total += len(df)
            df.to_csv('books/best_seller.csv', index=False)
    print('Number of books collected: ' + str(total))